# ** AI & Cognition Project**
# Web Engine
 
 5 DS 1 - 
 Group 1:

*   Med Anas FATTOUM
*   Iskander REGAIEG
*   Youssef Aziz ZGHAL
*   Haroun ELLEUCH
*   Saida MAJBOUR
*   Nadhir BOUHAOUALA


In [1]:
import pandas as pd
from rdfpandas.graph import to_dataframe
import pandas as pd
import rdflib
from utils2 import *
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, OWL, RDF, RDFS, SKOS, VOID, XMLNS, XSD
from rdflib import URIRef, BNode, Literal, Namespace, Graph, ConjunctiveGraph
from rdflib.extras import describer


In [2]:
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bouzm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
g = Graph()
g.parse('ontology.owl', format='nt')

<Graph identifier=N936a658154b443698f473a81a5c67537 (<class 'rdflib.graph.Graph'>)>

In [4]:
df = to_dataframe(g)
docs = df[ df['rdfs:comment{Literal}'].notna() ].reset_index()

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
data_input = docs['rdfs:comment{Literal}']

for i in range(len(docs["rdf:type{URIRef}[0]"])):
    for title in str(docs["rdf:type{URIRef}[0]"][i]).split("#")[1:]:
        title = title.replace('_',' ')
        data_input[i] += " "+title
    for title in str(docs["index"][i]).split("#")[1:]:
        title = title.replace('_',' ')
        data_input[i] += " "+title

X = vectorizer.fit_transform(data_input)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
df_vocab = pd.DataFrame(X, index=vectorizer.get_feature_names())
X.shape

C:\Users\bouzm\.conda\envs\ml\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(1255, 49)

In [6]:
def get_similar_parts(q, df):
    q = [q]
    q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
    sim = {}
    for i in range(df.shape[1]):
        sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)

    # Sort the values 
    sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    # Print the articles and their similarity values
    result = []
    for k, v in sim_sorted:
         if v >= 0.085 :
            title = ''
            sub_title = ''
            sub_sub_title = ''
            parag = docs.iloc[k,:]['rdfs:comment{Literal}']
            uri = str(docs.iloc[k,:]["rdf:type{URIRef}[0]"]).split("#")
            
            if (len(uri) > 0):
                if(uri[0]!='nan'):
                    title = str(docs.iloc[k,:]["rdf:type{URIRef}[0]"]).split("#")[1]
                    sub_sub_title = ""
                    if len(str(docs.iloc[k,:]["rdf:type{URIRef}[0]"]).split("#")) > 1:
                        sub_sub_title = str(docs.iloc[k,:]["rdf:type{URIRef}[0]"]).split("#")[2]
            if len(str(docs.iloc[k,:]["index"]).split("#")) >= 1:
                sub_title = docs.iloc[k,:]["index"].split("#")[1] 
            result.append([title,sub_sub_title,sub_title,parag , v])
            #print()
    return result
        #title , sub_sub title = individuals , sub title , annotation , probailité

In [ ]:
import joblib
from flask import Flask
from flask import request
from flask_cors import CORS
import pandas as pd

app = Flask(__name__)
CORS(app)

@app.route("/annotate",methods=['GET'])
def index():
    argum=(list(request.args.values()))
    if(len(argum)!=1):
        return "passez un argument texte"
    l=((get_similar_parts(remove_stopwords(clean(argum[0])), df_vocab)))
    print(remove_stopwords(clean(argum[0])))
    if(l==None):
        return " ICI INDIV "
    if(len(l)==0):
        return " ICI INDIV "
    l=l[:13]
    liste_annot=[clean_annot(j[3]) for j in l]
    liste_indiv=[k[0].upper().replace('_',' ')+': '+k[1].replace('_',' ')+': '+k[2].replace('_',' ')+'?'+str(k[4]*100)[:4] for k in l ]
    annotation="##".join(liste_annot)
    indiv="##".join(liste_indiv)
    return(annotation+" ICI INDIV "+indiv)
    

app.run(host='localhost',port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Nov/2021 20:28:59] "GET /annotate?recherche=tool HTTP/1.1" 200 -


tool


127.0.0.1 - - [16/Nov/2021 20:29:02] "GET /annotate?recherche=pmi HTTP/1.1" 200 -


pmi


...

127.0.0.1 - - [16/Nov/2021 20:32:44] "GET /annotate?recherche=How%20many%20inputs%20does%20project%20management%20require HTTP/1.1" 200 -


many inputs does project management require


127.0.0.1 - - [16/Nov/2021 20:38:04] "GET /annotate?recherche=How%20many%20inputs%20does%20project%20risk%20management%20have%20%3F HTTP/1.1" 200 -


many inputs does project risk management have


127.0.0.1 - - [16/Nov/2021 20:38:38] "GET /annotate?recherche= HTTP/1.1" 200 -
